<a href="https://colab.research.google.com/github/muneeb-kk/All-Assignments/blob/main/Video_and_Audio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!pip install --upgrade --quiet google-genai

In [29]:
!pip install gTTS --quiet

In [30]:
!pip install playsound --quiet

In [31]:
from google.colab import userdata
GEMINI_API_KEY :str = userdata.get("GEMINI_API_KEY")
if GEMINI_API_KEY:
  print("API key Found")
else:
  print("API key not found")


API key Found


In [32]:
from google import genai
from google.genai import Client
client: Client = genai.Client(
    api_key = GEMINI_API_KEY,
)
model: str = "gemini-2.0-flash-exp"

In [33]:
video_file_name = "/content/1_minute_self-introduction_video(135).mp4"

In [36]:
import time

def upload_video(video_file_name):
  video_file = client.files.upload (path = video_file_name)
  while video_file.state == "PROCESSING":
    print("waiting for video to be proceed")
    time.sleep(10)
    video_file = client.files.get(name=video_file.name or "")

  if video_file.state =="FAILED":
    raise ValueError (video_file.state)
  print(f"Video processing complete" + (video_file.uri or ""))

  return video_file
  my_Video = upload_video(video_file_name)



In [45]:
from google.genai.types import Content, Part
prompt = """ For each scene in this video,
             generate caption that describe the scene along with any spoken text placed in quoatation marks.
             Place each other caption into an object with the time code of the caption in the video
         """
video = my_Video
response = client.models.generate_content(
    model = model,
    contents=[
        Content(
            role = "user",
            parts = [
                Part.from_uri(
                    file_uri =video.uri or "",
                    mime_type = video.mime_type or ""), # Correct the typo here: mimi_type -> mime_type

              ]),
        prompt,
    ]
)
print(response.text)


```json
{
  "00:00": "A man with dark hair is centered in the frame. He is wearing a black jacket and a pair of white and black headphones with red lights. He is talking in front of a teal background.  \"Hi, my name is Rejan Tolentino, you can call me John. I'm 24 years old. I have a three-year work experience with the BPO industry doing inbound and outbound calling.\"",
  "00:13": "\"First, I served as a technical consultant where my multitasking skills has been definitely enhanced, then eventually I switched to customer service wherein our integrity plays a very big role.\"",
   "00:27": "\"Definitely, sales must always be on point, ahead of the target, and at the same time making sure that each and every customer will always be satisfied with my assistance.\"",
 "00:39": "\"And on top of all these skills, I also have a talent with doing graphic designs, creating flyers, business cards, and social media advertisements, which will really be helpful for clients' marketing needs.\"",
  

In [48]:
from gtts import gTTS
from playsound import playsound
from IPython.display import Audio

tts = gTTS(text = "/""Hi, my name is Rejan Tolentino, you can call me John. I am 24 years old. I have a three-year work experience with the BPO industry doing inbound and outbound calling.First, I served as a technical consultant where my multitasking skills has been definitely enhanced, then eventually I switched to customer service wherein our integrity plays a very big role."
, lang = 'en')

with open ("output.mp3", "wb")as f:
  for chunk in tts.stream():
    f.write(chunk)

display(Audio("output.mp3", autoplay = True))